In [ ]:
!pip install transformers==4.30.2
!pip install datasets
!pip install accelerate==0.21.0
!pip install transformers[torch]
!pip install accelerate -U
!pip show transformers accelerate
!pip install rouge_score

  Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.3
    Uninstalling accelerate-0.29.3:
      Successfully uninstalled accelerate-0.29.3
  Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0
Name: transformers
Version: 4.30.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers

In [ ]:
from datasets import load_dataset
import numpy as np
import pandas as pd
import torch
import spacy
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

root_folder = "/content/drive/MyDrive/Judiciary_Dataset/Summarization"
train_text_folder = os.path.join(root_folder, "train", "case-description")
train_summary_folder = os.path.join(root_folder, "train", "hd")
valid_text_folder = os.path.join(root_folder, "valid", "case-description")
valid_summary_folder = os.path.join(root_folder, "valid", "hd")
test_text_folder = os.path.join(root_folder, "test", "case-description")
test_summary_folder = os.path.join(root_folder, "test", "hd")

# Function to load data from a folder
def load_data(text_folder, summary_folder):
    text_files = glob.glob(os.path.join(text_folder, "*.txt"))
    summary_files = glob.glob(os.path.join(summary_folder, "*.txt"))

    text_data = []
    summary_data = []

    for text_file, summary_file in zip(text_files, summary_files):
        with open(text_file, "r", encoding="utf-8") as text_f:
            text_content = text_f.read()
            text_data.append(text_content)

        with open(summary_file, "r", encoding="utf-8") as summary_f:
            summary_content = summary_f.read()
            summary_data.append(summary_content)

    return text_data, summary_data


# Load text and summary data into Python from the specified folders
train_text_data, train_summary_data = load_data(train_text_folder, train_summary_folder)
valid_text_data, valid_summary_data = load_data(valid_text_folder, valid_summary_folder)
test_text_data, test_summary_data = load_data(test_text_folder, test_summary_folder)

In [ ]:
from datasets import Dataset

# Create Datasets for each split
train_dataset = Dataset.from_dict({"text": train_text_data, "summary": train_summary_data})
valid_dataset = Dataset.from_dict({"text": valid_text_data, "summary": valid_summary_data})
test_dataset = Dataset.from_dict({"text": test_text_data, "summary": test_summary_data})


In [ ]:
from datasets import DatasetDict

# Sample dataset dictionary
data_dict = DatasetDict({
    "train": train_dataset,
    "valid": valid_dataset,
    "test": test_dataset
})

In [ ]:
data_dict['train']

Dataset({
    features: ['text', 'summary'],
    num_rows: 345
})

In [ ]:
dir(data_dict['train'])

['_TF_DATASET_REFS',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getitems__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_local_temp_path',
 '_check_index_is_initialized',
 '_data',
 '_estimate_nbytes',
 '_fingerprint',
 '_format_columns',
 '_format_kwargs',
 '_format_type',
 '_generate_tables_from_cache_file',
 '_generate_tables_from_shards',
 '_get_cache_file_path',
 '_get_output_signature',
 '_getitem',
 '_indexes',
 '_indices',
 '_info',
 '_map_single',
 '_new_dataset_with_indices',
 '_output_all_columns',
 '_push_parquet_shards_to_hub',
 '_save_to_disk_single',
 '_select_contiguous',
 '_select_wi

In [ ]:
data_dict['train'].data

InMemoryTable
text: string
summary: string
----
text: [["[Supreme Court of Pakistan]

Present: Faisal Arab and Mazhar Alam Khan Miankhel, JJ

MUHAMMAD AFZAL (DECD.) through L.Rs. and others ---Appellants Versus MUHAMMAD BASHIR and another ---Respondents

Civil Appeal No. 528-L of 2012, decided on 20th November, 2019.

(Against the judgment dated 17.09.2009 of the Lahore High Court, Lahore passed in R.S.A. No. 58 of 2008)

Hafiz Tassaduq Hussain v. Muhammad Din through Legal Heirs and others PLD 2011 SC 241; Mt. Hira Bibi and others v. Ram Hari Lal and others AIR 1925 Privy Council 203 and Sheik Kachu v. Muhammad Ali Mahmud AIR 1927 Calcutta 926 distinguished.

Maulvi Anwar-ul-Haq, Advocate Supreme Court and Syed Rifaqat Hussain Shah, Advocate-on-Record for Appellants.

Malik Noor Muhammad Awan, Advocate Supreme Court for Respondent No.1.

Date of hearing: 20th November, 2019.

ORDER

FAISAL ARAB, J.

The case of respondent No.1 in his suit for specific performance of contract was that 

In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint = 'nsi319/legal-led-base-16384'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
tokenized_sent=tokenizer(data_dict['train'][0:3]['text'])
from pprint import pprint
print(tokenized_sent)

{'input_ids': [[0, 10975, 41192, 21612, 837, 9, 1752, 742, 50118, 50118, 45195, 35, 274, 5655, 337, 4681, 8, 15775, 4759, 22223, 3338, 256, 811, 330, 4393, 6, 28229, 50118, 50118, 31502, 20241, 448, 2606, 17491, 1301, 2118, 36, 495, 43663, 1592, 149, 226, 4, 18591, 4, 8, 643, 21394, 19186, 1641, 3277, 44420, 24099, 20241, 448, 2606, 163, 13246, 5216, 8, 277, 21394, 44036, 2832, 4189, 50118, 50118, 41044, 16049, 440, 4, 195, 2517, 12, 574, 9, 1125, 6, 1276, 15, 291, 212, 759, 6, 954, 4, 50118, 50118, 1640, 41634, 5, 7579, 7000, 601, 4, 3546, 4, 23301, 9, 5, 10838, 1688, 755, 837, 6, 10838, 1688, 1595, 11, 248, 4, 104, 4, 250, 4, 440, 4, 4893, 9, 2266, 43, 50118, 50118, 725, 2001, 1210, 255, 44642, 257, 1343, 16656, 748, 4, 6728, 15757, 149, 10661, 91, 21098, 8, 643, 12901, 495, 1466, 4998, 35752, 131, 11319, 4, 289, 3578, 163, 17310, 8, 643, 748, 4, 3513, 2482, 118, 15667, 8, 643, 23464, 36248, 9522, 11454, 1080, 23041, 8, 264, 967, 229, 46035, 748, 4, 6728, 4110, 27002, 1906, 23464, 35

In [ ]:
def preprocess_function(examples):
    # Tokenize the inputs and labels
    model_inputs = tokenizer(examples['text'], max_length=1024, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['summary'], max_length=128, truncation=True, padding="max_length")

    # The model expects labels to be in its input
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = data_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/345 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/42 [00:00<?, ? examples/s]

Map:   0%|          | 0/42 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM

In [ ]:
model=AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
model

LEDForConditionalGeneration(
  (led): LEDModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): LEDEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): LEDLearnedPositionalEmbedding(16384, 768)
      (layers): ModuleList(
        (0-5): 6 x LEDEncoderLayer(
          (self_attn): LEDEncoderAttention(
            (longformer_self_attn): LEDEncoderSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_global): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): Linear(in_features=768, out_features=768, bias=True)
          )
     

In [ ]:
training_args = TrainingArguments(
    'my_trainer',                          #  Output directory where checkpoints and logs will be saved
    evaluation_strategy='epoch',           # Evaluate model at the end of each epoch
    save_strategy='epoch',                  # Save model checkpoint at the end of each epoch
    num_train_epochs=1,
)

In [ ]:
from rouge_score import rouge_scorer, scoring

# Function to compute metrics
def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    # Decode the predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them
    decoded_labels = [label if label != tokenizer.pad_token else "" for label in decoded_labels]

    # Initialize Rouge Scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    # Create aggregator to compute average
    aggregator = scoring.BootstrapAggregator()

    # Calculate scores for each pair and aggregate
    for ref, pred in zip(decoded_labels, decoded_preds):
        score = scorer.score(ref, pred)
        aggregator.add_scores(score)

    # Final scores
    result = aggregator.aggregate()
    # Convert to dictionary with required format
    return {
        "rouge1": result["rouge1"].mid.fmeasure,
        "rouge2": result["rouge2"].mid.fmeasure,
        "rougeL": result["rougeL"].mid.fmeasure
    }

In [ ]:
from transformers import Trainer

In [ ]:
trainer=Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['valid'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
trainer.save_model('my_model')

In [ ]:
.

In [ ]:
!pip install -U transformers datasets accelerate peft trl bitsandbytes wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting un

In [ ]:
import gc
import os

import torch
import wandb
from datasets import load_dataset
from google.colab import userdata
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import ORPOConfig, ORPOTrainer, setup_chat_format

wb_token = userdata.get('wandb')
wandb.login(key=wb_token)

SecretNotFoundError: Secret wandb does not exist.

In [ ]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

In [ ]:
# Model
base_model = "meta-llama/Meta-Llama-3-8B"
new_model = "OrpoLlama-3-8B"

# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)
model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)

In [ ]:
orpo_args = ORPOConfig(
    learning_rate=8e-6,
    beta=0.1,
    lr_scheduler_type="linear",
    max_length=1024,
    max_prompt_length=512,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    report_to="wandb",
    output_dir="./results/",
)

trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    tokenizer=tokenizer,
)
trainer.train()
trainer.save_model(new_model)

In [ ]:
.

In [ ]:
import pandas as pd
import numpy as np
import torch
import spacy
import glob

In [ ]:
# Load your legal documents and summaries
# ...

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset

# Assuming train_text_data, train_summary_data, valid_text_data, valid_summary_data, test_text_data, test_summary_data are your original datasets

# Select the first 50 samples for each split
num_samples = 25
train_dataset = Dataset.from_dict({"text": train_text_data[:num_samples], "summary": train_summary_data[:num_samples]})
valid_dataset = Dataset.from_dict({"text": valid_text_data[:num_samples], "summary": valid_summary_data[:num_samples]})
test_dataset = Dataset.from_dict({"text": test_text_data[:num_samples], "summary": test_summary_data[:num_samples]})


In [ ]:
# Preprocess data, perform chunking, and extract named entities
# ...

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from datasets import load_dataset

In [ ]:
nltk.download('punkt')

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-led-base-16384")

In [ ]:
def process_data_to_model_inputs(batch):
    # Tokenize the inputs and labels
    inputs = tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["summary"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    # Assign tokenized input IDs and attention masks
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # Create global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # Set the 0 index for all samples to 1 in global_attention_mask
    batch["global_attention_mask"][0][0] = 1

    # Assign tokenized output IDs
    batch["labels"] = outputs.input_ids

    # Ensure that the PAD token is ignored in the labels
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [ ]:
max_input_length = 8192
max_output_length = 512
batch_size = 1


train_dataset = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["text", "summary"],
)

val_dataset = valid_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["text", "summary"],
)

train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)
val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

In [ ]:
from transformers import AutoModelForSeq2SeqLM
import torch
from datasets import load_dataset, load_metric
from transformers import LEDTokenizer, LEDForConditionalGeneration

In [ ]:
!pip install rouge_score

In [ ]:
led = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-led-base-16384", gradient_checkpointing=True, use_cache=False)

# set generate hyperparameters
led.config.num_beams = 2
led.config.max_length = 512
led.config.min_length = 172 #100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

rouge = load_metric("rouge")

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)



    rouge1_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid
    rouge2_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid
    rougeL_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid


    return {
        "rouge1_precision": round(rouge1_output.precision, 4),
        "rouge1_recall": round(rouge1_output.recall, 4),
        "rouge1_fmeasure": round(rouge1_output.fmeasure, 4),
        "rouge2_precision": round(rouge2_output.precision, 4),
        "rouge2_recall": round(rouge2_output.recall, 4),
        "rouge2_fmeasure": round(rouge2_output.fmeasure, 4),
        "rougeL_precision": round(rougeL_output.precision, 4),
        "rougeL_recall": round(rougeL_output.recall, 4),
        "rougeL_fmeasure": round(rougeL_output.fmeasure, 4),
    }

In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [ ]:
import accelerate

accelerate.__version__

In [ ]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

In [ ]:
import torch
torch.use_deterministic_algorithms(True)

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
# enable fp16 apex training
training_args = Seq2SeqTrainingArguments(
    gradient_accumulation_steps = 2,
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=False,
    output_dir="./",
    logging_steps=5,
    eval_steps=10,
    save_steps=10,
    save_total_limit=2,
    #gradient_accumulation_steps=4,
    num_train_epochs=2,
)

In [ ]:
from rouge_score import rouge_scorer, scoring

# Function to compute metrics
def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    # Decode the predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them
    decoded_labels = [label if label != tokenizer.pad_token else "" for label in decoded_labels]

    # Initialize Rouge Scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    # Create aggregator to compute average
    aggregator = scoring.BootstrapAggregator()

    # Calculate scores for each pair and aggregate
    for ref, pred in zip(decoded_labels, decoded_preds):
        score = scorer.score(ref, pred)
        aggregator.add_scores(score)

    # Final scores
    result = aggregator.aggregate()
    # Convert to dictionary with required format
    return {
        "rouge1": result["rouge1"].mid.fmeasure,
        "rouge2": result["rouge2"].mid.fmeasure,
        "rougeL": result["rougeL"].mid.fmeasure
    }

In [ ]:
trainer=Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
import os
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [ ]:
progress_bar = tqdm(total=training_args.num_train_epochs, desc="Training")

In [ ]:
for epoch in range(training_args.num_train_epochs):
    # Training step
    trainer.train()

    # Update the progress bar
    progress_bar.update(1)

# Close the progress bar
progress_bar.close()

In [ ]:

model = led

In [ ]:
# Evaluate on the validation dataset
results = trainer.evaluate()

# Print ROUGE scores
print(results)


In [ ]:
.

In [ ]:
def identify_section_headers(text):
    # Tokenize the text into words
    tokenized_text = word_tokenize(text)

    # Define a regex pattern to match common section header formats
    section_header_pattern = r"^(?:Present:|Civil Appeals Nos.|JUDGMENT.|ORDER.) [IVXLCDM]+[.:]?\s|^[A-Z][A-Z\s\-.]+:"

    section_headers = []

    for token in tokenized_text:
        # Check if the token matches the section header pattern
        if re.match(section_header_pattern, token, re.IGNORECASE):
            section_headers.append(token)

    return section_headers

In [ ]:
for split in ["train", "valid", "test"]:
    for example in data_dict[split]:
        text = example["text"]
        section_headers = identify_section_headers(text)

        # Printing the identified section headers for each example
        print(f"Split: {split}")
        print(section_headers)

In [ ]:
import re

def identify_section_headers(text, section_patterns):
    section_headers = []

    for pattern in section_patterns:
        matches = re.search(pattern, text, re.IGNORECASE)
        if matches:
            section_headers.append(matches.group(0))

    return section_headers

In [ ]:
section_patterns = [
        r"Present*",
        #r"Civil Appeals Nos. .*",
        #r"JUDGMENT.*",
        #r"ORDER.*",
        #r"conclusion.*",
        #r"SUBMISSIONS OF THE LEARNED COUNSEL FOR THE PETITIONER/LANDLORD:.*",
        #r"SUBMISSIONS MADE BY THE LEARNED COUNSEL FOR THE LANDLORDS/PRIVATE RESPONDENTS:.*",
        #r"SUBMISSIONS IN REJOINDER BY THE LEARNED COUNSEL FOR THE PETITIONER:.*",
        #r"After obtaining written replies learned Family Court framed following consolidated issues:*",
        #r"Relief.*",
        #r"No order as to Costs.*",
        #r"Learned counsel for the petitioner relied upon following authorities:.*",
        #r"He relied upon following authorities:.*",
        #r"Heard and record perused..*",
        #r"[Emphasis provided].*"
    ]

In [ ]:
for split in ["train", "valid", "test"]:
    for i, example in enumerate(data_dict[split]):
        if i < 2:  # Limit to the first 5 samples
            text = example["text"]
            section_headers = identify_section_headers(text, section_patterns)

            # Printing the identified section headers for each example
            print(f"Split: {split}")
            print(section_headers)
            print('\n')

In [ ]:
.

In [ ]:
import re
from datasets import load_dataset

def identify_sections(legal_text):
    section_patterns = [
        r"Present:.*",
        r"Civil Appeals Nos. .*",
        r"JUDGMENT.*",
        r"ORDER.*",
        r"conclusion.*",
        r"SUBMISSIONS OF THE LEARNED COUNSEL FOR THE PETITIONER/LANDLORD:.*",
        r"SUBMISSIONS MADE BY THE LEARNED COUNSEL FOR THE LANDLORDS/PRIVATE RESPONDENTS:.*",
        r"SUBMISSIONS IN REJOINDER BY THE LEARNED COUNSEL FOR THE PETITIONER:.*",
        r"After obtaining written replies learned Family Court framed following consolidated issues:*",
        r"Relief.*",
        r"No order as to Costs.*",
        r"Learned counsel for the petitioner relied upon following authorities:.*",
        r"He relied upon following authorities:.*",
        r"Heard and record perused..*",
        r"[Emphasis provided].*"
    ]

    sections = []

    for pattern in section_patterns:
        matches = re.finditer(pattern, legal_text)
        for match in matches:
            sections.append(match.group(0))

    return sections

In [ ]:
# Loop through the examples in the dataset and identify sections in the text
for example in data_dict['train']:
    text = example['text']
    sections_found = identify_sections(text)

    # Print the identified sections for each example
    if sections_found:
        for section in sections_found:
            print(section)
        print()

In [ ]:
data_dict

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = data_dict.map(preprocess_function, batched = True)

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
batch_size = 8
learning_rate=2e-5
weight_decay=0.01
num_train_epochs=2

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors ='tf')

In [ ]:
train_dataset = tokenized_datasets['train'].to_tf_dataset(
    batch_size=batch_size,
    columns = ['input_ids','attention_mask','labels'],
    shuffle = True,
    collate_fn = data_collator,
)

test_dataset = tokenized_datasets['test'].to_tf_dataset(
    batch_size=batch_size,
    columns = ['input_ids','attention_mask','labels'],
    shuffle = True,
    collate_fn = data_collator,
)

In [ ]:
from transformers import AdamWeightDecay
import tensorflow as tf

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=0.01)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(x=train_dataset, validation_data=test_dataset, epochs=num_train_epochs)

In [ ]:
# Fine-tune abstractive summarization model
# ...

# Fine-tune entailment model
# ...

# Evaluation using entailment module
# ...

# Generate summaries using the trained model
# ...
